Import the libraries

In [1]:
import numpy as np
import sympy as sp
import cvxpy as cp
import itertools
from scipy.linalg import null_space
from scipy.linalg import sqrtm
from numpy.linalg import inv
from numpy.linalg import det
from dataclasses import dataclass
from tqdm.notebook import tqdm

The tolerance for numerical computation, set to 1e-10 by default.

In [2]:
a_tol = 1e-10 # The absolute tolerance
r_tol = 1e-7  # The relative tolerance

Data Models

In [3]:
@dataclass
class Word_Bis:               # The class of bisectors in the symmetric space $X_3$. A list of Word_Bis models describes the bisectors defining the Dirichlet-Selber domain.
    word: np.ndarray          # A matrix $g$ in $SL(3,R)$, typically a word in given generators
    bis: np.ndarray           # A normal vector (as a 3*3 matrix) of the Selberg bisector $Bis(X, g.X)$

@dataclass
class Poly_Face:              # The class of faces of polytopes in $X_3$. A list of Poly_Face models describes the polytope structure of the Dirichlet-Selber domain.
    equs: list[int]           # A list of bisectors indices (in the list of the accompanying Word_Bis models) whose intersection is the minimal plane containing the face.
    codim: int                # The codimension (5 - dim) of the face.
    subfaces: list[int]       # A list of face indices that are proper subfaces of the current face.
    sample_point: np.ndarray  # A point in $X_3$ (as a 3*3 matrix) lying in the interior of the current face.

@dataclass
class Ridge_Cycle:            # The class describing a ridge-cycle of a Dirichlet-Selberg domain.
    ridge: list[int]          # A list of ridge indices (in the list of the accompanying Poly_Face models), for ridges $r_0, r_1, r_2,...$ in the same ridge cycle.
    pairing: list[int]        # A list of word indices (in the list of the accompanying Word_Bis models), each word $g_i$ sends $r_i$ to $r_{i+1}$.

Generic Helpers

In [4]:
def in_span(vectors, vec):
    """
    Parameters
    ----------
    vectors: list of np.ndarray
        a list of arrays describing a linearly independent collection of vectors.
    vec: np.ndarray
        an array describing a vector of the same dimension.

    Returns
    ----------
    is_in_span: bool
        True if `vec` lies in the span of `vectors`, up to tolerance.
    """
    if len(vectors) == 0:
        return np.linalg.norm(vec) < a_tol             # The first vector (if nonzero) is independent on its own
    matrix = np.array(vectors).T                       # Stack the current independent vectors into a matrix, so each column is a vector    
    projection = matrix @ np.linalg.pinv(matrix) @ vec # Compute the projection of the new vector onto the space spanned by the existing vectors
    residual = vec - projection                        # Compute the difference (residual) between the new vector and its projection
    return np.linalg.norm(residual) < a_tol            # The new vector is dependent to the existing ones if the residual is smaller than the threshold

def extract_basis(vectors):
    """
    Parameters
    ----------
    vectors: list of np.ndarray

    Returns
    ----------
    indep_vectors: list of np.ndarray
        a linearly independent sub-list of `vectors` (same span), up to tolerance.
    """
    indep_vectors = []
    for vector in vectors:
        if not in_span(indep_vectors, vector):
            indep_vectors.append(vector)
    return indep_vectors

def same_span(vectors_A, vectors_B):
    """
    Parameters
    ----------
    vectors_A: list of np.ndarray
    vectors_B: list of np.ndarray

    Returns
    ----------
    is_same_span: bool
        True if span(vectors_A) == span(vectors_B), up to tolerance.
    """
    rank_A = len(extract_basis(vectors_A))                   # Check if they define the same plane by a rank argument
    rank_B = len(extract_basis(vectors_B))
    rank_AplusB = len(extract_basis(vectors_A + vectors_B))
    return (rank_A == rank_AplusB) and (rank_B == rank_AplusB)

def symm_to_vec(mat, scale = 1):
    """
    Flatten a symmetric 3×3 `mat` into a 6-vector `vec`
    Parameters
    ----------
    mat: np.ndarray
    scale: np.float64

    Returns
    ----------
    vec: np.ndarray
    """
    vec = [mat[0][0], mat[1][1], mat[2][2], scale * mat[0][1], scale * mat[0][2], scale * mat[1][2]]
    return vec

def vec_to_symm(vec, scale = 1):
    """
    Reassemble a 6-vector `vec` into 3×3 symmetric matrix `mat`
    Parameters
    ----------
    vec: np.ndarray
    scale: np.float64

    Returns
    ----------
    mat: np.ndarray
    """
    mat = np.array([[vec[0], scale * vec[3], scale * vec[4]],
                     [scale * vec[3], vec[1], scale * vec[5]],
                     [scale * vec[4], scale * vec[5], vec[2]]])
    return mat

def is_positive_definite(mat):
    """
    Parameters
    ----------
    mat: np.ndarray

    Returns
    ----------
    is_positive_definite: bool
        Return True if symmetric matrix `mat` is strictly positive‐definite (all eigenvalues exceed `a_tol`).
    """
    if not isinstance(mat, np.ndarray):
        raise ValueError("Input must be a numpy array.")
    if mat.shape[0] != mat.shape[1]:
        return False                                  # The matrix is not square
    try:
        min_diag = np.min(np.linalg.eigvalsh(mat))    # Find the smallest eigenvalue
        return min_diag > a_tol                       # Positive definite if it is positive (concerning the tolerance).
    except np.linalg.LinAlgError:
        return False                                  # Not positive definite

Question-specific Helpers

In [5]:
def orth_complement(matrices):
    """
    Given a list of symmetric 3×3 `mats`, return a basis for their trace-orthogonal complement
    Parameters
    ----------
    matrices: list of np.ndarray

    Returns
    ----------
    orth_matrices: list of np.ndarray
    """
    vectors = [symm_to_vec(matrix) for matrix in matrices]
    vectors = extract_basis(vectors)
    orth_vectors = null_space(np.array(vectors)).T.tolist()
    orth_matrices = [np.array(vec_to_symm(orth_vector, 1/2)) for orth_vector in orth_vectors]
    return orth_matrices

def sympy_to_cvxpy(mat_sym, free_syms):
    """
    Convert a SymPy matrix `mat_sym` with variables in free_syms into a CVXPY expression
    Parameters
    ----------
    mat_sym: sp.Matrix
    free_syms: tuple of sp.symbols

    Returns
    ----------
    mat_cvx: cp.Expression
    """
    n_vars = len(free_syms)
    rows, cols = mat_sym.shape                                                                # Get the matrix shape
    coeff_matrices = [np.zeros((rows, cols), dtype=np.float64) for _ in range(n_vars + 1)]  # Initialize the coefficient matrices
    for i in range(rows):
        for j in range(cols):                 
            expr = sp.expand(mat_sym[i, j])                                                   # Decompose the matrix into entries
            if expr.is_Add:                 
                terms = expr.as_ordered_terms()                                             # Convert the entry into a list of summands
            else:
                terms = [expr]                                                              # Simply wrap the entry if it is a single term
            for term in terms:
                found = False
                for idx, sym in enumerate(free_syms):
                    if term.has(sym):
                        coeff = term.coeff(sym)
                        coeff_matrices[idx + 1][i, j] = float(coeff)                        # Add the coefficient of a certain variable to the corresponding matrix
                        found = True
                        break
                if not found:
                    coeff_matrices[0][i, j] = float(term)                                   # Add the constant term to the zeroth matrix
    x_cvx = cp.Variable(n_vars)                                                             # Define the cvxpy variables
    mat_cvx = coeff_matrices[0] + sum(x_cvx[i]*coeff_matrices[i+1] for i in range(n_vars))    # Combine the coefficient matrices into a cvxpy matrix with variables
    return mat_cvx

def perp_vectors(vec):
    """
    Parameters
    ----------
    vec: list of np.float64
        A 3-vector with at least one positive & one negative component

    Returns
    ----------
    u_vec: list of np.float64
        A positive-definite perpendicular vector
    w_vec: list of np.float64
        An indefinite perpendicular vector
    """
    positive_indices = [i for i, x in enumerate(vec) if x > a_tol]
    negative_indices = [i for i, x in enumerate(vec) if x < -a_tol]
    if not positive_indices or not negative_indices:
        return None, None                 # Our program does not focus on definite input vectors
    positive_index = positive_indices[0]  # Find the first positive component index of the vector
    negative_index = negative_indices[0]  # Find the first negative component index of the vector
    v0 = vec[positive_index]       # The positive one becomes v0
    v1 = vec[negative_index]       # The negative one becomes v1
    v2 = vec[3 - positive_index - negative_index] # The remaining becomes v2
    if v2<0:                              # Compute the perpendicular vectors in the new order
        u_sorted = [-v1/(np.sqrt(v0**2 + v1**2)) - v2/(np.sqrt(v0**2 + v2**2)), v0/(np.sqrt(v0**2 + v1**2)), v0/(np.sqrt(v0**2 + v2**2))]
        w_sorted = [-v1/(np.sqrt(v0**2 + v1**2)) + v2/(np.sqrt(v0**2 + v2**2)), v0/(np.sqrt(v0**2 + v1**2)), -v0/(np.sqrt(v0**2 + v2**2))]
    else:
        u_sorted = [-v1/(np.sqrt(v0**2 + v1**2)), v0/(np.sqrt(v0**2 + v1**2)) + v2/(np.sqrt(v1**2 + v2**2)), -v1/(np.sqrt(v1**2 + v2**2))]
        w_sorted = [-v1/(np.sqrt(v0**2 + v1**2)), v0/(np.sqrt(v0**2 + v1**2)) - v2/(np.sqrt(v1**2 + v2**2)), v1/(np.sqrt(v1**2 + v2**2))]
    u_vec = [u_sorted[[positive_index, negative_index, 3 - positive_index - negative_index].index(i)] for i in range(3)] # Permute the vector back to the original order
    w_vec = [w_sorted[[positive_index, negative_index, 3 - positive_index - negative_index].index(i)] for i in range(3)]
    return u_vec, w_vec

def find_psd_combination(mat_expr, free_syms):
    """
    Solve a small CVXPY problem to find a positive-definite linear combination of `mat_expr` in sp.Matrix with variables in free_syms
    Parameters
    ----------
    mat_expr: sp.Matrix
    free_syms: tuple of sp.symbols

    Returns
    ----------
    psd_combination: Optional[np.ndarray]
    """
    t = cp.Variable()
    x = cp.Variable(len(free_syms))
    mat_cvx = sympy_to_cvxpy(mat_expr, free_syms)                     # The combination of certain matrices
    constraints = [mat_cvx - t*np.eye(3) >> 0, x >= -1, x <= 1] # Our question only concerns coefficients lying between -1 and 1
    prob = cp.Problem(cp.Maximize(t), constraints)        # Find maximal t such that M-tI is positive definite
    prob.solve(solver=cp.SCS,eps=a_tol,max_iters=50000)     # Add verbose=True if needed
    if prob.value > 100*a_tol:
        return mat_cvx.value                                    # The positive definite linear combination with maximized least eigenvalue
    else:
        return None                                       # No positive definite linear combinations

def riemannian_angle(equ_1, equ_2, base):
    """
    Compute the Riemannian angle between two hyperplanes (normals `equ_1`, `equ_2`) at point `base` in X₃.
    Parameters
    ----------
    equ_1: np.ndarray
    equ_2: np.ndarray
    base: np.ndarray

    Returns
    ----------
    angle: np.float64
    """
    comp_1 = base @ equ_1
    comp_2 = base @ equ_2
    angle_cos = - (np.trace(comp_1 @ comp_2))/(np.sqrt((np.trace(comp_1 @ comp_1)) * (np.trace(comp_2 @ comp_2))))
    angle = np.arccos(angle_cos)   # Formula is discussed in my paper
    return angle

def perturb_within_plane(mat, plane_eqs, new_eq):
    """
    Given PD `mat` on the intersection of `plane_eqs`, perturb it slightly across `new_eq` while preserving PD and original constraints.
    Parameters
    ----------
    mat: np.ndarray
    plane_eqs: list of np.ndarray
    new_eq: np.ndarray

    Returns
    ----------
    proj_mat: np.ndarray
    """
    mat = np.array(mat)                                                           # make the matrices numpy for safety reason
    plane_eqs = [np.array(equ) for equ in plane_eqs]
    new_eq = np.array(new_eq)
    if not is_positive_definite(mat):
        raise ValueError("perturb_within_plane: Input must be positive definite.")
    else:
        mat_sqrt = sqrtm(mat)                                                     # Congruence so the input matrix is taken to the origin
        plane_eqs_trans = [mat_sqrt @ equ @ mat_sqrt for equ in plane_eqs]
        new_eq_trans = mat_sqrt @ new_eq @ mat_sqrt
        orth_equs = orth_complement(plane_eqs_trans)
        orth_vectors = [symm_to_vec(equ, np.sqrt(2)) for equ in orth_equs]    # Convert from matrices to vectors
        new_vector = symm_to_vec(new_eq_trans, np.sqrt(2))
        orth_vectors_mat = np.array(orth_vectors).T                                  # Stack the current independent vectors into a matrix, each column is a vector
        coeffs = list(np.linalg.pinv(orth_vectors_mat) @ new_vector)                 # Project the new vector to the existing ones
        proj_trans = sum(coeff*equ for coeff, equ in zip(coeffs, orth_equs)) # This linear combination lies on the desired plane while keeps away from the new hyperplane
        proj_mat = mat_sqrt @ proj_trans @ mat_sqrt                          # Take the matrix back
        while not is_positive_definite(proj_mat):
            proj_mat = 0.5*proj_mat + 0.5*mat                                    # Go back toward the original matrix if the new one is indefinite
        proj_mat = proj_mat/((det(proj_mat)) ** (1/3))                            # Unitize the matrix with respect to the determinant
        return proj_mat

def maps_plane_equal(old_eqs, new_eqs, isom):
    """
    Parameters
    ----------
    old_eqs: list of np.ndarray
    new_eqs: list of np.ndarray
    isom: np.ndarray

    Returns
    ----------
    is_mapped: bool
        True if applying `isom` in sends the hyperplane system `old_eqs` to `new_eqs` (i.e. spaces coincide).
    """
    mapped_eqs = [inv(isom) @ mat @ inv(isom.T) for mat in old_eqs] # The normal matrices for the mapped plane
    mapped_vecs = [symm_to_vec(mat) for mat in mapped_eqs]          # Convert from matrices to vectors
    new_vecs = [symm_to_vec(mat) for mat in new_eqs]
    return same_span(mapped_vecs, new_vecs)

Special Utilities

In [6]:
def distinct_matrix_products(gens, max_length):
    """
    Enumerate all distinct products in {gens} U {gens⁻¹} up to length `max_length` and avoiding back‐tracking
    Parameters
    ----------
    gens: list of np.ndarray
    max_length: int

    Returns
    ----------
    mat_products: list of np.ndarray
    """
    gens = [np.array(mat) for mat in gens] # Make sure they are numpy arrays
    k = len(gens)
    G = gens + [inv(mat) for mat in gens]
    inv_idx = lambda j: (j + k) % (2*k)
    mat_products = [np.eye(gens[0].shape[0])]  # The list of matrices and last-index-list, List[np.ndarray]
    frontier = {0: set()} # Dict[int, Set[int]]
    for _ in range(max_length):
        next_frontier = {}  # Dict[int, Set[int]] 
        for ind in frontier:            # A tuple of word and a possible last-index
            for j, mat in enumerate(G):        # A tuple of generator and its index to be added
                if inv_idx(j) in frontier[ind]:
                    continue                   # Avoid backtracking
                N = mat_products[ind] @ mat                    # If not, add the generator to the end of the word
                matched = False
                for S_ind in range(len(mat_products)):
                    if np.allclose(N, mat_products[S_ind], atol = a_tol, rtol = r_tol): # The word is already there; if it is in next_frontier, add a last-index
                        matched = True
                        if S_ind in next_frontier:
                            next_frontier[S_ind].add(j)
                        break
                if not matched:
                    mat_products.append(N)
                    N_ind = len(mat_products) - 1
                    next_frontier[N_ind] = set()
                    next_frontier[N_ind].add(j)
        frontier = next_frontier
        if not frontier:
            break
    mat_products.sort(key=lambda M: np.trace(M.T @ M)) # sort by Frobenius norm
    return mat_products

def find_feasible_point(plane_eqs):
    """
    Check if the intersection of the given hyperplane constraints is non-empty. Return a feasible point or None.
    Parameters
    ----------
    plane_eqs: list of np.ndarray

    Returns
    ----------
    feasible_point: Optional[np.ndarray]
    """
    vectors = [symm_to_vec(equ) for equ in plane_eqs]
    independent_vectors = extract_basis(vectors)
    indep_matrix = [vec_to_symm(vec) for vec in independent_vectors] # Extract a linearly independent sublist
    indep_matrix_sp = [sp.Matrix(mat) for mat in indep_matrix]
    A = indep_matrix[0]
    eigenvalues, eigenvectors = np.linalg.eigh(A)
    ev_orth, ev_orth_neg = perp_vectors(eigenvalues)
    if ev_orth is None:
        feasible_point = None                           # No feasible points if A is semi-definite
    else:
        n = len(indep_matrix)
        Q = eigenvectors
        D_orth = np.diag(ev_orth)
        if n == 1:
            feasible_point = Q @ D_orth @ Q.T           # If n = 1, Find the feasible point from the perpendicular vector of the eigenvalue
        else:
            linearized_matrix = [Q.T @ mat @ Q for mat in indep_matrix]
            linearized_matrix_sp = [sp.Matrix(mat) for mat in linearized_matrix]
            x_v = sp.symbols('x1:5')
            x_v_new = x_v                                # The necessary variables
            diag_max = [a + abs(b) for a, b in zip(ev_orth, ev_orth_neg)]
            D_orth_neg = [np.diag(ev_orth_neg),\
                        vec_to_symm(np.sqrt(diag_max[0]*diag_max[1]) * np.eye(6)[:, 3]),\
                        vec_to_symm(np.sqrt(diag_max[0]*diag_max[2]) * np.eye(6)[:, 4]),\
                        vec_to_symm(np.sqrt(diag_max[1]*diag_max[2]) * np.eye(6)[:, 5])]
            D_orth_sp = sp.Matrix(D_orth)
            D_orth_neg_sp = [sp.Matrix(mat) for mat in D_orth_neg]
            matrix_comb = sum((var * mat for mat, var in zip(D_orth_neg_sp, x_v)), start=D_orth_sp)
            for i in range(1,n):                        # Remove extra variables
                trace_matrix_prod = (matrix_comb * linearized_matrix_sp[i]).trace().expand()
                trace_coeffs = {var: trace_matrix_prod.coeff(var) for var in x_v}
                if all(abs(coeff) < a_tol for coeff in trace_coeffs.values()): # If a nonzero constant appears, return to false since it is surely empty.
                    return None                                                # To avoid dead loop, this case has to return on its own
                max_var = max(trace_coeffs, key=lambda v: abs(trace_coeffs[v]))
                x_sol = sp.solve(trace_matrix_prod, max_var)[0]                # Solve f = 0 for max_var
                x_v_new = tuple(var for var in x_v_new if var != max_var)      # Drop max_var from x_v_new
                matrix_comb = matrix_comb.subs(max_var,x_sol)
            if n == 5:                                   # If all variables are removed, the feasible point is derived from matrix_comb
                D = np.array(matrix_comb).astype(np.float64)
                if is_positive_definite(D):
                    feasible_point = Q @ D @ Q.T
                else:
                    feasible_point = None
            else:                                        # Otherwise, use cvxpy to find the positive-definite linear combination
                poly_comb = matrix_comb.det()
                poly_comb_coeff = poly_comb.as_coefficients_dict()
                all_zero = all(abs(coef) < a_tol for coef in poly_comb_coeff.values())
                if all_zero:
                    feasible_point = None
                else:
                    D = find_psd_combination(matrix_comb, x_v_new)
                    if D is None:
                        feasible_point = None
                    else:
                        feasible_point = Q @ D @ Q.T
    if feasible_point is not None:
        if not is_positive_definite(feasible_point):     # In corner cases, the matrix cvxpy produced may not be positive definite
            feasible_point = None
        else:
            feasible_point = np.array(feasible_point)
            feasible_point = feasible_point/((det(feasible_point)) ** (1/3))
    return feasible_point

def add_facet_to_domain(wbs, faces, new_wb):
    """
    Insert `new_wb` into the Dirichlet-Selberg domain structure (`wbs`, `faces`), updating the face lists in place.
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face
    new_wb: Word_Bis

    Returns
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face
    """
    new_vec = symm_to_vec(new_wb.bis)
    face_conds = [0]*len(faces)                            # Assign a condition number (1 to 6, initialized with 0) to each face
    for j in range(len(face_conds)):
        face_equs = [wbs[ind].bis for ind in faces[j].equs]
        face_vecs = [symm_to_vec(equ) for equ in face_equs]
        if in_span(extract_basis(face_vecs), new_vec):     # It is type 1 if the equations defining F_j span the new equation
            face_conds[j] = 1
        elif faces[j].subfaces == []:                      # The case if face is minimal
            if find_feasible_point(face_equs + [new_wb.bis]) is not None: # It is type 6 if the new hyperplane intersects with the minimal face
                face_conds[j] = 6
            else:                                                         # It is type 2 or 4 if the new hyperplane does not intersect with the minimal face
                face_sample_point = faces[j].sample_point
                if np.trace(face_sample_point @ new_wb.bis) > 0:
                    face_conds[j] = 2
                else:
                    face_conds[j] = 4
        else:                                              # The case if face is not minimal (has subfaces)
            face_subfaces = faces[j].subfaces
            face_subfaces_temp = [face_conds[ind] for ind in face_subfaces]
            if 6 in face_subfaces_temp:                    # Any subface is of type 6 -> type 6
                face_conds[j] = 6
            elif {2, 3} & set(face_subfaces_temp) and {4, 5} & set(face_subfaces_temp): # One subface is of type 2 or 3, another subface is of type 4 or 5 -> type 6
                face_conds[j] = 6
            elif {1, 3, 5} & set(face_subfaces_temp):      # One subface is of type 1, 3 or 5 -> type 3 or 5
                face_sample_point = faces[j].sample_point
                if np.trace(face_sample_point @ new_wb.bis) > 0:
                    face_conds[j] = 3
                else:
                    face_conds[j] = 5
            else:                                                     # All subfaces are of type 2 or 4
                if find_feasible_point(face_equs + [new_wb.bis]) is not None: # If the new hyperplane intersects the span of the face
                    face_inters_sample_point = find_feasible_point(face_equs + [new_wb.bis])
                    face_subfaces_equs = []
                    for ind in face_subfaces:
                        if faces[ind].codim == faces[j].codim + 1:
                            face_subfaces_equs_temp = [elem for elem in faces[ind].equs if elem not in faces[j].equs]
                            face_subfaces_equs[:] = list(set(face_subfaces_equs) | set(face_subfaces_equs_temp))
                    face_conds[j] = 6                                  # The case below does not happen -> 6
                    for ind in face_subfaces_equs:
                        if np.trace(face_inters_sample_point @ wbs[ind].bis) < 0:
                            face_conds[j] = face_subfaces_temp[0]      # If any side separates the sample point from the face -> 2 or 4
                            break
                else:                                                  # If the new hyperplane does not intersect the span of the face -> 2 or 4
                    face_conds[j] = face_subfaces_temp[0]

    ind_remove_list = [j for j in range(len(face_conds)) if face_conds[j] in [4, 5]]
    for j in sorted(ind_remove_list, reverse=True):     # Faces of type 4 or 5: will be deleted
        del faces[j]
        del face_conds[j]
    for j in range(len(face_conds)):                    # The presence of these faces as subfaces of the other is also erased.
        faces[j].subfaces = [ind for ind in faces[j].subfaces if ind not in ind_remove_list]
        List_subfaces_temp = []
        for ind in faces[j].subfaces:
            decrease = sum(1 for val in ind_remove_list if val < ind)
            List_subfaces_temp.append(ind - decrease)
        faces[j].subfaces = List_subfaces_temp.copy()
    for j in range(len(face_conds)):                    # Faces of type 1: the new equation will be added
        if face_conds[j] == 1:
            faces[j].equs.append(len(wbs))
        elif face_conds[j] == 6:                        # Faces of type 6
            new_face_equs = faces[j].equs + [len(wbs)]  # a new face will born
            new_face_codim = faces[j].codim + 1         # Codimension of the new face
            new_face_subfaces = [ind for ind in faces[j].subfaces if face_conds[ind] == 1]
            for ind in faces[j].subfaces:
                if ind < len(face_conds):
                    if face_conds[ind] == 6:
                        faces[j].subfaces.append(faces[ind].subfaces[-1])
                        new_face_subfaces.append(faces[ind].subfaces[-1])
            faces[j].subfaces.append(len(faces))        # Subfaces for both old and new faces
            if len(new_face_subfaces) == 0:             # Sample point for the new face
                face_equs = [wbs[ind].bis for ind in faces[j].equs]
                new_face_sample_point = find_feasible_point(face_equs + [new_wb.bis])
            elif len(new_face_subfaces) >= 2:           # >=2 subfaces: barycenter
                new_face_sample_point = sum((faces[ind].sample_point for ind in new_face_subfaces), np.zeros((3, 3)))
                new_face_sample_point = new_face_sample_point/((det(new_face_sample_point)) ** (1/3))
            else:                                       # Exactly 1 subface: pertube the sample point of the subface
                face_equs = [wbs[ind].bis for ind in faces[j].equs]
                subface_equ_ind = next((ind for ind in faces[new_face_subfaces[0]].equs if ind not in set(new_face_equs)), None)
                subface_equ = wbs[subface_equ_ind].bis
                subface_sample_point = faces[new_face_subfaces[0]].sample_point
                new_face_sample_point = perturb_within_plane(subface_sample_point, face_equs + [new_wb.bis], subface_equ)
            if np.trace(faces[j].sample_point @ new_wb.bis) < np.sqrt(a_tol):   # Sample point for the old face: pertube the sample point of the new face
                face_equs = [wbs[ind].bis for ind in faces[j].equs]
                old_face_sample_point = perturb_within_plane(new_face_sample_point, face_equs, new_wb.bis)
                while any(ind not in faces[j].equs and np.trace(wb.bis @ faces[j].sample_point) > a_tol\
                          and np.trace(wb.bis @ old_face_sample_point) < a_tol for ind, wb in enumerate(wbs)): # Check if this point is inside the polytope
                    old_face_sample_point = 0.5*(old_face_sample_point + new_face_sample_point)
                temporary_sample_point = 0.5*(old_face_sample_point + new_face_sample_point)
                temporary_sample_point = temporary_sample_point/((det(temporary_sample_point)) ** (1/3))
                faces[j].sample_point = temporary_sample_point
            faces.append(Poly_Face(new_face_equs, new_face_codim, new_face_subfaces, np.array(new_face_sample_point)))  # Save the new face to faces
    wbs.append(new_wb)                                                           # Save the new equation to bises_active

    equ_remove_list = list(range(len(wbs)))                                      # Remove the unnecessary equations
    for j in range(len(faces)):
        if faces[j].codim == 1:
            equ_remove_list = [ind for ind in equ_remove_list if ind != faces[j].equs[0]]
    for j in sorted(equ_remove_list, reverse=True):
        del wbs[j]
    for j in range(len(faces)):
        faces[j].equs = [ind for ind in faces[j].equs if ind not in equ_remove_list]
        List_equs_temp = []
        for ind in faces[j].equs:
            decrease = sum(1 for val in equ_remove_list if val < ind)
            List_equs_temp.append(ind - decrease)
        faces[j].equs = List_equs_temp.copy()
    faces_indexed = [(i, face) for i, face in enumerate(faces)]
    faces_indexed.sort(key=lambda obj: obj[1].codim, reverse=True)                # Sort the faces by their dimension (small to large)
    index_mapping = {old_index: new_index for new_index, (old_index, _) in enumerate(faces_indexed)}
    for _, face in faces_indexed:
        face.subfaces = [index_mapping[ind] for ind in face.subfaces]
    faces = [face for _, face in faces_indexed]
    return wbs, faces

def are_faces_paired(wbs, faces, i_old, i_new, isom):
    """
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face
    i_old: int
    i_new: int
    isom: np.ndarray

    Returns
    ----------
    are_paired: bool
        True if the facet indexed `i_old` and `i_new` are paired by word `isom`.
    """
    if faces[i_old].codim != faces[i_new].codim:
        return False
    if len(faces[i_old].subfaces) != len(faces[i_new].subfaces):
        return False
    old_face_equations = [wbs[ind].bis for ind in faces[i_old].equs]
    new_face_equations = [wbs[ind].bis for ind in faces[i_new].equs]
    if not maps_plane_equal(old_face_equations, new_face_equations, isom):
        return False
    if len(faces[i_old].subfaces) == 0:
        return True
    cod = faces[i_old].codim
    old_facets = [j for j in faces[i_old].subfaces if faces[j].codim == cod + 1]
    new_facets = [k for k in faces[i_new].subfaces if faces[k].codim == cod + 1]
    if len(old_facets) != len(new_facets):
        return False
    for j in old_facets:
        if not any(are_faces_paired(wbs, faces, j, k, isom) for k in new_facets):
            return False
    return True

def find_unpaired_ridges(wbs, faces):
    """
    Return the list of ridge indices with associated facet indices that remain unpaired in the current (pre-exact) Dirichlet-Selberg domain.
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face

    Returns
    ----------
    unpaired_ridges: list of [int, int]
    """
    facet_indices = [i for i in range(len(faces)) if faces[i].codim == 1]
    unpaired_ridges = []
    for i in facet_indices:
        i_ridges = [j for j in faces[i].subfaces if faces[j].codim == 2]
        i_pair = next((i_0 for i_0 in facet_indices if np.all(np.abs(wbs[faces[i].equs[0]].word @ wbs[faces[i_0].equs[0]].word\
                                                                     - np.eye(3))<a_tol)), None)
        if i_pair != None:
            i_pair_ridges = [j_0 for j_0 in faces[i_pair].subfaces if faces[j_0].codim == 2]
            for j in i_ridges:
                j_equations = [wbs[ind].bis for ind in faces[j].equs]
                j_pair = next((j_0 for j_0 in i_pair_ridges if maps_plane_equal(j_equations,\
                                        [wbs[ind].bis for ind in faces[j_0].equs],wbs[faces[i].equs[0]].word)), None)
                if j_pair is None:
                    mat = (wbs[faces[i].equs[0]].word).T @ faces[j].sample_point @ wbs[faces[i].equs[0]].word
                    if all(np.trace(mat @ wbs[ind].bis)> a_tol for ind in range(len(wbs)) if ind not in faces[i_pair].equs):
                        unpaired_ridges.append([i, j])
    return unpaired_ridges

def find_path_word(wbs, faces, dest):
    """
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face
    dest: np.ndarray

    Returns
    ----------
    path_word: list of int
        The product of facet pairings of indices in `path_word` takes `dest` into the Dirichlet‐Selberg domain.
    """
    if not is_positive_definite(dest):
        raise ValueError("find_path_word: destination point must be positive definite.")
    dest = dest/(det(dest) ** (1/3))                            # Normalize the destination point
    center = next(face.sample_point for face in faces if face.codim == 0)  # The center is the sample point of the polytope itself
    if all(np.trace(dest @ wb.bis)>-a_tol for wb in wbs):                  # If the destination is already in the Dirichlet-Selberg domain
        return []
    meet_ind = max(range(len(wbs)), key = lambda i: -(np.trace(dest @ wbs[i].bis))/(np.trace(center @ wbs[i].bis)))
    meet_word = wbs[meet_ind].word                                             # The word corresponding to the first bisector meeting the ray from the center towards the destination
    new_dest = meet_word.T @ dest @ meet_word                               # Take the destination closer to the center
    new_path = find_path_word(wbs, faces, new_dest)
    new_path.insert(0, meet_ind)
    return new_path


Core Solvers

In [7]:
def compute_word_bisectors(gens, max_length, center):
    """
    For each non‐trivial word w (|w|≤max_length) in the group generated by `gens` that moves `center`, compute the bisector Bis(center, w.center).
    Parameters
    ----------
    gens: list of np.ndarray
    max_length: int
    center: np.ndarray

    Returns
    ----------
    wbs: list of Word_Bis
    """
    words = distinct_matrix_products(gens, max_length)
    wbs = [Word_Bis(word, np.array(word) @ inv(np.array(center)) @ np.array(word).T - inv(np.array(center)))\
           for word in words]                                                # Definition of Selberg bisectors
    wbs_filtered = [wb for wb in wbs if not np.all(np.abs(wb.bis)<a_tol)]      # Exclude bisectors with zero normal matrices
    return wbs_filtered

def compute_selberg_domain(gens, center, length_1, length_2 = None, max_loops = 0):
    """
    Build the Dirichlet‐Selberg domain: first extend all words up to `length_1`, then iteratively pair ridges using words up to `length_2`, for at most `max_loops`.
    Parameters
    ----------
    gens: list of np.ndarray
    center: np.ndarray
    length_1: int
    length_2: int
    max_loops: int

    Returns
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face
    """
    if length_2 is None:
        length_2 = length_1
    wbs_1 = compute_word_bisectors(gens, length_1, center)
    wbs_2 = compute_word_bisectors(gens, length_2, center)
    wbs = []                                           # Initialize the word-bisectors used in the polytope
    faces = [Poly_Face([], 0, [], np.array(center))]     # Initialize the polytope, which is just the entire space X_3
    for i in tqdm(range(len(wbs_1)), desc ="Adding words to the Dirichlet-Selberg domain"):
        wbs, faces = add_facet_to_domain(wbs, faces, wbs_1[i]) # Add the i-th word-bisector to the polytope
    for _ in tqdm(range(max_loops), desc ="Adding more words to eliminate unpaired ridges"):
        unpaired_ridges = find_unpaired_ridges(wbs, faces)
        if not unpaired_ridges:                           # Stop searching if all ridges are paired
            break
        else:
            facet_indices = [i for i in range(len(faces)) if faces[i].codim == 1]
            min_dist = np.inf
            new_wb = None                                     # Searching for the candidate bisector closest to the origin
            for i, j in unpaired_ridges:
                i_pair = next((i_0 for i_0 in facet_indices if np.all(np.abs(wbs[faces[i].equs[0]].word @ wbs[faces[i_0].equs[0]].word\
                                                                     - np.eye(3))<a_tol)), None)
                j_equations = [wbs[ind].bis for ind in faces[j].equs]
                i_pair_equation = wbs[faces[i_pair].equs[0]].bis
                for k in range(len(wbs_2)):
                    if np.trace(wbs_2[k].bis @ np.array(center)) < min_dist:
                        candidate_equations = [i_pair_equation, wbs_2[k].bis]
                        if maps_plane_equal(j_equations, candidate_equations, wbs[faces[i].equs[0]].word):
                            min_dist = np.trace(wbs_2[k].bis @ np.array(center))
                            new_wb = wbs_2[k]              # Update the desired bisector if a smaller distance is detected
            if new_wb is not None:
                wbs, faces = add_facet_to_domain(wbs, faces, new_wb)
    return wbs, faces

def polytope_exactness(wbs, faces):
    """
    If the domain is exact, return a dict: {"facet indices": [], "paired_indices": []}, else return None.
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face

    Returns
    ----------
    pairing_dict: dict of int
    """
    facet_indices = [i for i in range(len(faces)) if faces[i].codim == 1] # Get a list of facets
    pairing_dict = {}
    for i in facet_indices:
        i_pair = next((j for j in facet_indices if are_faces_paired(wbs, faces,\
                                                                  i, j, wbs[faces[i].equs[0]].word)), None)
        if i_pair is None:
            return None                                           # False if facets are not paired
        else:
            pairing_dict[i] = i_pair
    return pairing_dict                                          # Corresponding facets are canonically paired

def compute_ridge_cycles(wbs, faces):
    """
    For an exact polytope, return a list of ridge cycles.
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face

    Returns
    ----------
    ridge_cycles: list of Ridge_Cycle
    """
    pairing_dict = polytope_exactness(wbs, faces)
    if pairing_dict is None:                  # Ridges cycles are defined only for exact polytopes 
        return None
    facet_indices = list(pairing_dict)
    all_ridge_indices = [i for i in range(len(faces)) if faces[i].codim == 2]
    ridge_cycles = []                                        # Initialize the list of ridge cycles
    for i in facet_indices:
        ridge_indices = [j for j in faces[i].subfaces if j in all_ridge_indices]
        for j in ridge_indices:                                  # Consider the index pair for a facet and a ridge of it
            if any(j in ridge_cycle.ridge for ridge_cycle in ridge_cycles):
                continue                                         # Case if it is already in a ridge cycle
            current_ridge = j
            current_facet = i                                # Chasing the ridges along the cycle
            current_pairing = faces[current_facet].equs[0]
            ridge_cycle = Ridge_Cycle([current_ridge], [current_pairing])
            for _ in range(2*len(all_ridge_indices)):        # Ridge cycles will not be too long
                mapped_facet = pairing_dict[current_facet]
                new_ridge_indices = [new_j for new_j in faces[mapped_facet].subfaces if new_j in all_ridge_indices]
                new_ridge = next((new_j for new_j in new_ridge_indices if \
                                    are_faces_paired(wbs, faces, current_ridge, new_j, wbs[current_pairing].word)), None)
                new_facet = next(new_i for new_i in facet_indices if new_ridge in faces[new_i].subfaces and new_i != mapped_facet)
                new_pairing = faces[new_facet].equs[0]
                if new_ridge == ridge_cycle.ridge[0] and new_pairing == ridge_cycle.pairing[0]:
                    ridge_cycles.append(ridge_cycle)      # Add the ridge cycle to ridge_cycles if a full cycle is obtained
                    break
                ridge_cycle.ridge.append(new_ridge)       # Shift to the next ridge and facet if the cycle is not completed
                ridge_cycle.pairing.append(new_pairing)
                current_ridge = new_ridge
                current_facet = new_facet
                current_pairing = new_pairing
    return ridge_cycles

def compute_angle_sum(wbs, faces, ridge_cycle):
    """
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face
    ridge_cycle: Ridge_Cycle

    Returns
    ----------
    angle_sum_quotient: Optional[int]
        Return k if the angle sum around `ridge_cycle` is 2π/k; else None.
    """
    angle_sum = 0                                           # Initialize the angle sum
    point = faces[ridge_cycle.ridge[0]].sample_point # The base point of the first ridge is selected to be the given sample point
    for i in range(len(ridge_cycle.ridge)):
        first_bis = faces[ridge_cycle.ridge[i]].equs[0]
        second_bis = faces[ridge_cycle.ridge[i]].equs[1]
        angle = riemannian_angle(wbs[first_bis].bis, wbs[second_bis].bis, point) # Compute the Riemannian angle between the bisectors
        angle_sum = angle_sum + angle                       # Add this to the angle sum
        word = wbs[ridge_cycle.pairing[i]].word          
        point = word.T @ point @ word                       # Shift to the paired base point of the next ridge
    quotient = 2*np.pi/angle_sum                            # Check if the quotient of the angle sum with 2pi is a natural number
    angle_sum_quotient = round(quotient)
    if abs(quotient - angle_sum_quotient)> 100*a_tol:
        return None
    else:
        return angle_sum_quotient

def is_word_recovered(wbs, faces, isom):
    """
    Check if the group element `isom` is realized by a facet pairing in the domain.
    Parameters
    ----------
    wbs: list of Word_Bis
    faces: list of Poly_Face
    isom: np.ndarray

    Returns
    ----------
    is_recovered: bool
    """
    isom = isom/(det(isom)**(1/3))
    center = next(face.sample_point for face in faces if face.codim == 0)  # The center is the sample point of the polytope itself
    dest_point = isom.T @ center @ isom
    word = find_path_word(wbs, faces, dest_point)
    for ind in word:
        isom = isom @ wbs[ind].word
    if np.all(np.abs(isom - np.eye(3))<a_tol):
        return True
    else:
        return False

Demo Program: A congruence subgroup of level-two in SL(3,Z) (may take several minutes to finish)

In [8]:
generators = []
for i, j in itertools.permutations(range(3), 2):
    generator = np.eye(3)
    generator[i, j] = 2
    generators.append(generator)
center = np.eye(3)
wbs, faces = compute_selberg_domain(generators, center, 1, 2, 20)

Adding words to the Dirichlet-Selberg domain:   0%|          | 0/12 [00:00<?, ?it/s]

/usr/local/python/3.12.1/lib/python3.12/site-packages/cvxpy/problems/problem.py:1510: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Adding more words to eliminate unpaired ridges:   0%|          | 0/20 [00:00<?, ?it/s]

Demo Program: A lattice group in SL(3,Z[1/2]) with 5-simplex fundamental domain (quick)

In [17]:
generator = np.array([[1/2, 1/2, 0],
        [1/2, -1/2, 1],
        [1/2, -1/2, -1]])
generators = [generator, generator[[1, 2, 0]][:, [1, 2, 0]]]
center = np.eye(3)
wbs, faces = compute_selberg_domain(generators, center, 1, 2, 20)

Adding words to the Dirichlet-Selberg domain:   0%|          | 0/4 [00:00<?, ?it/s]

Adding more words to eliminate unpaired ridges:   0%|          | 0/20 [00:00<?, ?it/s]

Analyze the result

In [15]:
# 1) Count faces by codimension
for codim, name in [(1,"facets"), (2,"ridges"), (3,"peaks"), (4,"edges"), (5,"vertices")]:
    count = sum(1 for f in faces if f.codim == codim)
    print(f"Number of {name}: {count}")

# 2) Exactness & ridge‑cycles
pairing_dict = polytope_exactness(wbs, faces)
if pairing_dict is None:
    print("Domain is NOT exact; increase word lengths.")
else:
    cycles = compute_ridge_cycles(wbs, faces)
    for i, cycle in enumerate(cycles):
        k = compute_angle_sum(wbs, faces, cycle)
        print(f"Cycle {i}: ridge indices={cycle.ridge}, angle‑sum divisor={k}")

# 3) Generator recovery
for i, gen in enumerate(generators):
    if is_word_recovered(wbs, faces, gen):
        path = find_path_word(wbs, faces, gen.T @ center @ gen)
        print(f"Generator #{i} is recovered via pairings {path}")
    else:
        print(f"Generator #{i} is NOT recovered")

Number of facets: 6
Number of ridges: 15
Number of peaks: 16
Number of edges: 0
Number of vertices: 0
Cycle 0: ridge indices=[16, 17, 21], angle‑sum divisor=2
Cycle 1: ridge indices=[19, 24, 27], angle‑sum divisor=2
Cycle 2: ridge indices=[22, 28, 30], angle‑sum divisor=2
Cycle 3: ridge indices=[26, 18, 25], angle‑sum divisor=1
Cycle 4: ridge indices=[20, 29, 23], angle‑sum divisor=2
Generator #0 is recovered via pairings [2]
Generator #1 is recovered via pairings [3]


Verify that the sample points are taken correctly

In [16]:
# 1) Maximum absolute trace over each face's own bisectors
max_on_plane = max(
    abs(np.trace(wbs[i].bis @ face.sample_point))
    for face in faces
    for i    in face.equs
)
print(
    "Max |tr(A·X)| on each face’s plane:", 
    max_on_plane
)

# 2) Minimum trace over all bisectors *not* defining that face
all_indices = set(range(len(wbs)))
min_off_plane = min(
    np.trace(wbs[j].bis @ face.sample_point)
    for face in faces
    for j    in (all_indices - set(face.equs))
)
print(
    "Min  tr(A·X) off each face’s plane:", 
    min_off_plane
)

Max |tr(A·X)| on each face’s plane: 2.7755575615628914e-15
Min  tr(A·X) off each face’s plane: 0.0875728471814019
